In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd

df= pd.read_csv('/content/drive/MyDrive/neural_network/sentiment140_vader4_sinlematizar.csv')
df

,polarity,id,date,query,user,text,vader_polarity,preprocesado,word_count,preprocesado_constopwords,label,preprocesado_sinlematizar
0,negative,2184288131,Mon Jun 15 15:41:28 PDT 2009,NO_QUERY,Zelus,I want my e-mail with my tracking info for my ...,0.0772,want email track info new iphone,12,i want my email with my track info for my new ...,0,i want my email with my tracking info for my n...
1,negative,1834602716,Mon May 18 03:33:33 PDT 2009,NO_QUERY,S810uk,Is not happy with the weather I'm off to the ...,-0.4585,happy weather im museum later boy sock culture...,27,is not happy with the weather im off to the mu...,0,is not happy with the weather im off to the mu...
2,positive,1976167179,Sat May 30 16:44:04 PDT 2009,NO_QUERY,MicheleBlueston,@cindyscottday and forgot...you got kiddies ho...,0.5106,forgotyou get kiddy home still free agent take...,16,and forgotyou get kiddy home still free agent ...,1,and forgotyou got kiddies home still free agen...
3,positive,2047727017,Fri Jun 05 14:05:11 PDT 2009,NO_QUERY,adrian_x,@iota @brandie will ALWAYS be number two in MY...,0.0772,always number two book,8,will always be number two in m book,1,will always be number two in m book
4,positive,2014391411,Wed Jun 03 01:22:32 PDT 2009,NO_QUERY,muchloveanna,"@cjmccoll no problem gosh, her work is amazing!",0.7469,problem josh work amaze,7,no problem josh her work be amaze,1,no problem josh her work is amazing
...,...,...,...,...,...,...,...,...,...,...,...,...
121099,negative,2175983387,Mon Jun 15 02:34:22 PDT 2009,NO_QUERY,zapakitul,Back from school! Two more weeks of school he...,-0.6467,back school two week school hate highschool,13,back from school two more week of school here ...,0,back from school two more weeks of school here...
121100,positive,2185123417,Mon Jun 15 16:55:17 PDT 2009,NO_QUERY,dreacham,@lauren_nichelle i'll be sure to let daddy kno...,0.7639,ill sure let daddy know hell really happy,11,ill be sure to let daddy know hell be really h...,1,ill be sure to let daddy know hell be really h...
121101,positive,1834315088,Mon May 18 02:27:47 PDT 2009,NO_QUERY,Cherry_Gryffon,@erniehalter YAYYY new country invasion XD *do...,0.6739,yyy new country invasion xd wi helmet,8,yyy new country invasion xd don wi helmet,1,yyy new country invasion xd dons wi helmet
121102,positive,1676509898,Fri May 01 22:15:41 PDT 2009,NO_QUERY,tessthetraveler,just came back from op shopping in wynnum ther...,0.6184,come back op shopping wynnum many 2nd hand sto...,24,just come back from op shopping in wynnum ther...,1,just came back from op shopping in wynnum ther...


In [ ]:
from sklearn.model_selection import train_test_split

# Dividir el conjunto de datos en entrenamiento y prueba
X = df['preprocesado_sinlematizar']
y = df['label']

# Primero dividimos en entrenamiento (80%) y prueba (20%)
X_temp, X_test, y_temp, y_test = train_test_split(X, y, test_size=0.2, random_state=34, stratify=y)

# Luego dividimos el conjunto temporal en entrenamiento (80% de 80%) y validación (20% de 80%)
X_train, X_val, y_train, y_val = train_test_split(X_temp, y_temp, test_size=0.2, random_state=34, stratify=y_temp)


In [ ]:
import numpy as np
embedding_matrix_x_train= np.load('/content/drive/MyDrive/neural_network/embeddings_matrix_validation.npy')

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Configuración de Tokenizer
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train.astype(str))
sequences = tokenizer.texts_to_sequences(X_train.astype(str))

max_length = 25
X_train_def = pad_sequences(sequences, maxlen=max_length, padding='post')


embedding_dim = 400
word_index = tokenizer.word_index
num_words = len(word_index) + 1

In [ ]:
# Convertir X_test a secuencias numéricas
sequences_test = tokenizer.texts_to_sequences(X_test.astype(str))  # Convertir a secuencias
X_test_def = pad_sequences(sequences_test, maxlen=max_length, padding='post')  # Asegurarse de que tengan la misma longitud que max_len

# Convertir X_val a secuencias numéricas
sequences_val = tokenizer.texts_to_sequences(X_val.astype(str))  # Convertir a secuencias
X_val_def = pad_sequences(sequences_val, maxlen=max_length, padding='post')  # Asegurar la longitud adecuada


In [ ]:
import numpy as np
import random
import tensorflow as tf

seed=42
np.random.seed(seed)
random.seed(seed)
tf.random.set_seed(seed)

In [ ]:

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Conv1D, MaxPooling1D, Flatten, LSTM, Dense, Dropout, BatchNormalization, Bidirectional
from tensorflow.keras.callbacks import EarlyStopping

# Parámetros
max_len = X_train_def.shape[1]
seed=42
np.random.seed(seed)
random.seed(seed)
tf.random.set_seed(seed)

model = Sequential()
model.add(Embedding(input_dim=num_words, output_dim=embedding_dim, weights=[embedding_matrix_x_train], trainable=False, input_length=max_len))
model.add(Dropout(0.25))

model.add(LSTM(64))


model.add(Dense(64, activation='relu'))

model.add(Dense(1, activation='sigmoid'))

from tensorflow.keras.optimizers import Adam
optimizer = Adam(learning_rate=0.0005)  # Reducir el learning rate
model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy']) # Función de pérdida binaria

# Definir Early Stopping
early_stopping = EarlyStopping(monitor='val_accuracy', patience=3, restore_best_weights=True)


# Entrenamiento con Early Stopping usando conjuntos de validación específicos
history = model.fit(
    X_train_def, y_train,
    epochs=20,
    batch_size=32,
    validation_data=(X_val_def, y_val),  # Especifica los conjuntos de validación
    callbacks=[early_stopping]
)

Epoch 1/20
2423/2423 ━━━━━━━━━━━━━━━━━━━━ 18s 6ms/step - accuracy: 0.7520 - loss: 0.4862 - val_accuracy: 0.8284 - val_loss: 0.3734
Epoch 2/20
2423/2423 ━━━━━━━━━━━━━━━━━━━━ 20s 6ms/step - accuracy: 0.8257 - loss: 0.3755 - val_accuracy: 0.8410 - val_loss: 0.3589
Epoch 3/20
2423/2423 ━━━━━━━━━━━━━━━━━━━━ 15s 6ms/step - accuracy: 0.8428 - loss: 0.3460 - val_accuracy: 0.8479 - val_loss: 0.3390
Epoch 4/20
2423/2423 ━━━━━━━━━━━━━━━━━━━━ 16s 6ms/step - accuracy: 0.8514 - loss: 0.3288 - val_accuracy: 0.8519 - val_loss: 0.3316
Epoch 5/20
2423/2423 ━━━━━━━━━━━━━━━━━━━━ 22s 7ms/step - accuracy: 0.8617 - loss: 0.3133 - val_accuracy: 0.8543 - val_loss: 0.3374
Epoch 6/20
2423/2423 ━━━━━━━━━━━━━━━━━━━━ 20s 7ms/step - accuracy: 0.8675 - loss: 0.2993 - val_accuracy: 0.8540 - val_loss: 0.3327
Epoch 7/20
2423/2423 ━━━━━━━━━━━━━━━━━━━━ 20s 6ms/step - accuracy: 0.8722 - loss: 0.2898 - val_accuracy: 0.8579 - val_loss: 0.3256
Epoch 8/20
2423/2423 ━━━━━━━━━━━━━━━━━━━━ 20s 6ms/step - accuracy: 0.8796 - loss: 0

In [ ]:
sequences = tokenizer.texts_to_sequences(X_test)
X_test_def = pad_sequences(sequences, maxlen=max_length, padding='post')
# Evaluación en el conjunto de test
test_loss, test_accuracy = model.evaluate(X_test_def, y_test, batch_size=64)

print(f"Pérdida en el conjunto de test: {test_loss}")
print(f"Precisión en el conjunto de test: {test_accuracy}")

379/379 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.8548 - loss: 0.3274
Pérdida en el conjunto de test: 0.3289847671985626
Precisión en el conjunto de test: 0.8565707206726074


In [ ]:

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Conv1D, MaxPooling1D, Flatten, LSTM, Dense, Dropout, BatchNormalization, Bidirectional
from tensorflow.keras.callbacks import EarlyStopping

# Parámetros
max_len = X_train_def.shape[1]
seed=42
np.random.seed(seed)
random.seed(seed)
tf.random.set_seed(seed)

model = Sequential()
model.add(Embedding(input_dim=num_words, output_dim=embedding_dim, weights=[embedding_matrix_x_train], trainable=False, input_length=max_len))
model.add(Dropout(0.25))

model.add(LSTM(64))
model.add(Dropout(0.5))

model.add(Dense(64, activation='relu'))

model.add(Dense(1, activation='sigmoid'))

from tensorflow.keras.optimizers import Adam
optimizer = Adam(learning_rate=0.0005)  # Reducir el learning rate
model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy']) # Función de pérdida binaria

# Definir Early Stopping
early_stopping = EarlyStopping(monitor='val_accuracy', patience=3, restore_best_weights=True)


# Entrenamiento con Early Stopping usando conjuntos de validación específicos
history = model.fit(
    X_train_def, y_train,
    epochs=20,
    batch_size=32,
    validation_data=(X_val_def, y_val),  # Especifica los conjuntos de validación
    callbacks=[early_stopping]
)

Epoch 1/20
2423/2423 ━━━━━━━━━━━━━━━━━━━━ 18s 7ms/step - accuracy: 0.7379 - loss: 0.4985 - val_accuracy: 0.8283 - val_loss: 0.3745
Epoch 2/20
2423/2423 ━━━━━━━━━━━━━━━━━━━━ 16s 7ms/step - accuracy: 0.8236 - loss: 0.3805 - val_accuracy: 0.8373 - val_loss: 0.3618
Epoch 3/20
2423/2423 ━━━━━━━━━━━━━━━━━━━━ 21s 7ms/step - accuracy: 0.8413 - loss: 0.3516 - val_accuracy: 0.8493 - val_loss: 0.3405
Epoch 4/20
2423/2423 ━━━━━━━━━━━━━━━━━━━━ 20s 7ms/step - accuracy: 0.8486 - loss: 0.3358 - val_accuracy: 0.8532 - val_loss: 0.3308
Epoch 5/20
2423/2423 ━━━━━━━━━━━━━━━━━━━━ 16s 7ms/step - accuracy: 0.8575 - loss: 0.3208 - val_accuracy: 0.8559 - val_loss: 0.3423
Epoch 6/20
2423/2423 ━━━━━━━━━━━━━━━━━━━━ 16s 7ms/step - accuracy: 0.8646 - loss: 0.3076 - val_accuracy: 0.8595 - val_loss: 0.3309
Epoch 7/20
2423/2423 ━━━━━━━━━━━━━━━━━━━━ 22s 7ms/step - accuracy: 0.8704 - loss: 0.2978 - val_accuracy: 0.8622 - val_loss: 0.3214
Epoch 8/20
2423/2423 ━━━━━━━━━━━━━━━━━━━━ 18s 7ms/step - accuracy: 0.8748 - loss: 0

In [ ]:
sequences = tokenizer.texts_to_sequences(X_test)
X_test_def = pad_sequences(sequences, maxlen=max_length, padding='post')
# Evaluación en el conjunto de test
test_loss, test_accuracy = model.evaluate(X_test_def, y_test, batch_size=64)

print(f"Pérdida en el conjunto de test: {test_loss}")
print(f"Precisión en el conjunto de test: {test_accuracy}")

379/379 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8570 - loss: 0.3240
Pérdida en el conjunto de test: 0.3242965340614319
Precisión en el conjunto de test: 0.8592130541801453


In [ ]:

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Conv1D, MaxPooling1D, Flatten, LSTM, Dense, Dropout, BatchNormalization, Bidirectional
from tensorflow.keras.callbacks import EarlyStopping

# Parámetros
max_len = X_train_def.shape[1]
seed=42
np.random.seed(seed)
random.seed(seed)
tf.random.set_seed(seed)

model = Sequential()
model.add(Embedding(input_dim=num_words, output_dim=embedding_dim, weights=[embedding_matrix_x_train], trainable=False, input_length=max_len))
model.add(Dropout(0.25))

model.add(LSTM(64))
model.add(Dropout(0.3))

model.add(Dense(64, activation='relu'))

model.add(Dense(1, activation='sigmoid'))

from tensorflow.keras.optimizers import Adam
optimizer = Adam(learning_rate=0.0005)  # Reducir el learning rate
model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy']) # Función de pérdida binaria

# Definir Early Stopping
early_stopping = EarlyStopping(monitor='val_accuracy', patience=3, restore_best_weights=True)


# Entrenamiento con Early Stopping usando conjuntos de validación específicos
history = model.fit(
    X_train_def, y_train,
    epochs=20,
    batch_size=32,
    validation_data=(X_val_def, y_val),  # Especifica los conjuntos de validación
    callbacks=[early_stopping]
)

Epoch 1/20
2423/2423 ━━━━━━━━━━━━━━━━━━━━ 20s 7ms/step - accuracy: 0.7432 - loss: 0.4929 - val_accuracy: 0.8303 - val_loss: 0.3725
Epoch 2/20
2423/2423 ━━━━━━━━━━━━━━━━━━━━ 18s 6ms/step - accuracy: 0.8242 - loss: 0.3787 - val_accuracy: 0.8398 - val_loss: 0.3608
Epoch 3/20
2423/2423 ━━━━━━━━━━━━━━━━━━━━ 20s 6ms/step - accuracy: 0.8425 - loss: 0.3485 - val_accuracy: 0.8490 - val_loss: 0.3420
Epoch 4/20
2423/2423 ━━━━━━━━━━━━━━━━━━━━ 21s 6ms/step - accuracy: 0.8494 - loss: 0.3318 - val_accuracy: 0.8546 - val_loss: 0.3313
Epoch 5/20
2423/2423 ━━━━━━━━━━━━━━━━━━━━ 16s 6ms/step - accuracy: 0.8592 - loss: 0.3165 - val_accuracy: 0.8537 - val_loss: 0.3488
Epoch 6/20
2423/2423 ━━━━━━━━━━━━━━━━━━━━ 21s 7ms/step - accuracy: 0.8673 - loss: 0.3036 - val_accuracy: 0.8607 - val_loss: 0.3315
Epoch 7/20
2423/2423 ━━━━━━━━━━━━━━━━━━━━ 20s 6ms/step - accuracy: 0.8719 - loss: 0.2926 - val_accuracy: 0.8606 - val_loss: 0.3292
Epoch 8/20
2423/2423 ━━━━━━━━━━━━━━━━━━━━ 21s 7ms/step - accuracy: 0.8768 - loss: 0

In [ ]:
sequences = tokenizer.texts_to_sequences(X_test)
X_test_def = pad_sequences(sequences, maxlen=max_length, padding='post')
# Evaluación en el conjunto de test
test_loss, test_accuracy = model.evaluate(X_test_def, y_test, batch_size=64)

print(f"Pérdida en el conjunto de test: {test_loss}")
print(f"Precisión en el conjunto de test: {test_accuracy}")

379/379 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.8575 - loss: 0.3301
Pérdida en el conjunto de test: 0.33220455050468445
Precisión en el conjunto de test: 0.858511209487915


In [ ]:

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Conv1D, MaxPooling1D, Flatten, LSTM, Dense, Dropout, BatchNormalization, Bidirectional
from tensorflow.keras.callbacks import EarlyStopping

# Parámetros
max_len = X_train_def.shape[1]
seed=42
np.random.seed(seed)
random.seed(seed)
tf.random.set_seed(seed)

model = Sequential()
model.add(Embedding(input_dim=num_words, output_dim=embedding_dim, weights=[embedding_matrix_x_train], trainable=False, input_length=max_len))
model.add(Dropout(0.25))

model.add(LSTM(64))


model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

from tensorflow.keras.optimizers import Adam
optimizer = Adam(learning_rate=0.0005)  # Reducir el learning rate
model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy']) # Función de pérdida binaria

# Definir Early Stopping
early_stopping = EarlyStopping(monitor='val_accuracy', patience=3, restore_best_weights=True)

# Entrenamiento con Early Stopping usando conjuntos de validación específicos
history = model.fit(
    X_train_def, y_train,
    epochs=20,
    batch_size=32,
    validation_data=(X_val_def, y_val),  # Especifica los conjuntos de validación
    callbacks=[early_stopping]
)

Epoch 1/20
2423/2423 ━━━━━━━━━━━━━━━━━━━━ 18s 7ms/step - accuracy: 0.7468 - loss: 0.4966 - val_accuracy: 0.8276 - val_loss: 0.3782
Epoch 2/20
2423/2423 ━━━━━━━━━━━━━━━━━━━━ 20s 6ms/step - accuracy: 0.8247 - loss: 0.3809 - val_accuracy: 0.8387 - val_loss: 0.3608
Epoch 3/20
2423/2423 ━━━━━━━━━━━━━━━━━━━━ 16s 6ms/step - accuracy: 0.8423 - loss: 0.3519 - val_accuracy: 0.8464 - val_loss: 0.3400
Epoch 4/20
2423/2423 ━━━━━━━━━━━━━━━━━━━━ 22s 7ms/step - accuracy: 0.8505 - loss: 0.3345 - val_accuracy: 0.8505 - val_loss: 0.3325
Epoch 5/20
2423/2423 ━━━━━━━━━━━━━━━━━━━━ 19s 7ms/step - accuracy: 0.8591 - loss: 0.3188 - val_accuracy: 0.8553 - val_loss: 0.3347
Epoch 6/20
2423/2423 ━━━━━━━━━━━━━━━━━━━━ 22s 7ms/step - accuracy: 0.8661 - loss: 0.3059 - val_accuracy: 0.8580 - val_loss: 0.3275
Epoch 7/20
2423/2423 ━━━━━━━━━━━━━━━━━━━━ 17s 7ms/step - accuracy: 0.8711 - loss: 0.2954 - val_accuracy: 0.8591 - val_loss: 0.3227
Epoch 8/20
2423/2423 ━━━━━━━━━━━━━━━━━━━━ 17s 7ms/step - accuracy: 0.8773 - loss: 0

In [ ]:
sequences = tokenizer.texts_to_sequences(X_test)
X_test_def = pad_sequences(sequences, maxlen=max_length, padding='post')
# Evaluación en el conjunto de test
test_loss, test_accuracy = model.evaluate(X_test_def, y_test, batch_size=64)

print(f"Pérdida en el conjunto de test: {test_loss}")
print(f"Precisión en el conjunto de test: {test_accuracy}")

379/379 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8571 - loss: 0.3308
Pérdida en el conjunto de test: 0.3299297094345093
Precisión en el conjunto de test: 0.8588002324104309


In [ ]:

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Conv1D, MaxPooling1D, Flatten, LSTM, Dense, Dropout, BatchNormalization, Bidirectional
from tensorflow.keras.callbacks import EarlyStopping

# Parámetros
max_len = X_train_def.shape[1]
seed=42
np.random.seed(seed)
random.seed(seed)
tf.random.set_seed(seed)

model = Sequential()
model.add(Embedding(input_dim=num_words, output_dim=embedding_dim, weights=[embedding_matrix_x_train], trainable=False, input_length=max_len))
model.add(Dropout(0.25))

model.add(LSTM(128))
model.add(Dropout(0.5))

model.add(Dense(64, activation='relu'))

model.add(Dense(1, activation='sigmoid'))

from tensorflow.keras.optimizers import Adam
optimizer = Adam(learning_rate=0.0005)  # Reducir el learning rate
model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy']) # Función de pérdida binaria

# Definir Early Stopping
early_stopping = EarlyStopping(monitor='val_accuracy', patience=3, restore_best_weights=True)

# Entrenamiento con Early Stopping usando conjuntos de validación específicos
history = model.fit(
    X_train_def, y_train,
    epochs=20,
    batch_size=32,
    validation_data=(X_val_def, y_val),  # Especifica los conjuntos de validación
    callbacks=[early_stopping]
)

Epoch 1/20
2423/2423 ━━━━━━━━━━━━━━━━━━━━ 21s 8ms/step - accuracy: 0.7561 - loss: 0.4857 - val_accuracy: 0.8307 - val_loss: 0.3727
Epoch 2/20
2423/2423 ━━━━━━━━━━━━━━━━━━━━ 16s 7ms/step - accuracy: 0.8246 - loss: 0.3773 - val_accuracy: 0.8427 - val_loss: 0.3602
Epoch 3/20
2423/2423 ━━━━━━━━━━━━━━━━━━━━ 20s 6ms/step - accuracy: 0.8458 - loss: 0.3456 - val_accuracy: 0.8516 - val_loss: 0.3345
Epoch 4/20
2423/2423 ━━━━━━━━━━━━━━━━━━━━ 22s 7ms/step - accuracy: 0.8543 - loss: 0.3274 - val_accuracy: 0.8548 - val_loss: 0.3281
Epoch 5/20
2423/2423 ━━━━━━━━━━━━━━━━━━━━ 19s 6ms/step - accuracy: 0.8622 - loss: 0.3106 - val_accuracy: 0.8543 - val_loss: 0.3526
Epoch 6/20
2423/2423 ━━━━━━━━━━━━━━━━━━━━ 16s 6ms/step - accuracy: 0.8714 - loss: 0.2961 - val_accuracy: 0.8588 - val_loss: 0.3307
Epoch 7/20
2423/2423 ━━━━━━━━━━━━━━━━━━━━ 16s 6ms/step - accuracy: 0.8776 - loss: 0.2819 - val_accuracy: 0.8566 - val_loss: 0.3345
Epoch 8/20
2423/2423 ━━━━━━━━━━━━━━━━━━━━ 20s 6ms/step - accuracy: 0.8851 - loss: 0

In [ ]:

# Evaluación en el conjunto de test
test_loss, test_accuracy = model.evaluate(X_test_def, y_test, batch_size=64)

print(f"Pérdida en el conjunto de test: {test_loss}")
print(f"Precisión en el conjunto de test: {test_accuracy}")

379/379 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8555 - loss: 0.3435
Pérdida en el conjunto de test: 0.3441172242164612
Precisión en el conjunto de test: 0.8578506112098694


In [ ]:

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Conv1D, MaxPooling1D, Flatten, LSTM, Dense, Dropout, BatchNormalization, Bidirectional
from tensorflow.keras.callbacks import EarlyStopping

# Parámetros
max_len = X_train_def.shape[1]
seed=42
np.random.seed(seed)
random.seed(seed)
tf.random.set_seed(seed)

model = Sequential()
model.add(Embedding(input_dim=num_words, output_dim=embedding_dim, weights=[embedding_matrix_x_train], trainable=False, input_length=max_len))
model.add(Dropout(0.25))

model.add(LSTM(256))
model.add(Dropout(0.5))

model.add(Dense(64, activation='relu'))

model.add(Dense(1, activation='sigmoid'))

from tensorflow.keras.optimizers import Adam
optimizer = Adam(learning_rate=0.0005)  # Reducir el learning rate
model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy']) # Función de pérdida binaria

# Definir Early Stopping
early_stopping = EarlyStopping(monitor='val_accuracy', patience=3, restore_best_weights=True)

# Entrenamiento con Early Stopping usando conjuntos de validación específicos
history = model.fit(
    X_train_def, y_train,
    epochs=20,
    batch_size=32,
    validation_data=(X_val_def, y_val),  # Especifica los conjuntos de validación
    callbacks=[early_stopping]
)

Epoch 1/20
2423/2423 ━━━━━━━━━━━━━━━━━━━━ 22s 9ms/step - accuracy: 0.7541 - loss: 0.4867 - val_accuracy: 0.8355 - val_loss: 0.3667
Epoch 2/20
2423/2423 ━━━━━━━━━━━━━━━━━━━━ 42s 9ms/step - accuracy: 0.8289 - loss: 0.3741 - val_accuracy: 0.8453 - val_loss: 0.3552
Epoch 3/20
2423/2423 ━━━━━━━━━━━━━━━━━━━━ 36s 7ms/step - accuracy: 0.8464 - loss: 0.3426 - val_accuracy: 0.8559 - val_loss: 0.3296
Epoch 4/20
2423/2423 ━━━━━━━━━━━━━━━━━━━━ 22s 8ms/step - accuracy: 0.8557 - loss: 0.3219 - val_accuracy: 0.8582 - val_loss: 0.3296
Epoch 5/20
2423/2423 ━━━━━━━━━━━━━━━━━━━━ 19s 7ms/step - accuracy: 0.8671 - loss: 0.3032 - val_accuracy: 0.8610 - val_loss: 0.3516
Epoch 6/20
2423/2423 ━━━━━━━━━━━━━━━━━━━━ 19s 7ms/step - accuracy: 0.8755 - loss: 0.2846 - val_accuracy: 0.8607 - val_loss: 0.3441
Epoch 7/20
2423/2423 ━━━━━━━━━━━━━━━━━━━━ 21s 7ms/step - accuracy: 0.8849 - loss: 0.2656 - val_accuracy: 0.8570 - val_loss: 0.3390
Epoch 8/20
2423/2423 ━━━━━━━━━━━━━━━━━━━━ 17s 7ms/step - accuracy: 0.8950 - loss: 0

In [ ]:

# Evaluación en el conjunto de test
test_loss, test_accuracy = model.evaluate(X_test_def, y_test, batch_size=64)

print(f"Pérdida en el conjunto de test: {test_loss}")
print(f"Precisión en el conjunto de test: {test_accuracy}")

379/379 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8566 - loss: 0.3562
Pérdida en el conjunto de test: 0.35788002610206604
Precisión en el conjunto de test: 0.8581396341323853


In [ ]:

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Conv1D, MaxPooling1D, Flatten, LSTM, Dense, Dropout, BatchNormalization, Bidirectional
from tensorflow.keras.callbacks import EarlyStopping

# Parámetros
max_len = X_train_def.shape[1]
seed=42
np.random.seed(seed)
random.seed(seed)
tf.random.set_seed(seed)

model = Sequential()
model.add(Embedding(input_dim=num_words, output_dim=embedding_dim, weights=[embedding_matrix_x_train], trainable=False, input_length=max_len))
model.add(Dropout(0.25))

model.add(LSTM(64, return_sequences=True))
model.add(Dropout(0.5))
model.add(LSTM(64))
model.add(Dense(64, activation='relu'))

model.add(Dense(1, activation='sigmoid'))

from tensorflow.keras.optimizers import Adam
optimizer = Adam(learning_rate=0.0005)  # Reducir el learning rate
model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy']) # Función de pérdida binaria

# Definir Early Stopping
early_stopping = EarlyStopping(monitor='val_accuracy', patience=3, restore_best_weights=True)

# Entrenamiento con Early Stopping usando conjuntos de validación específicos
history = model.fit(
    X_train_def, y_train,
    epochs=20,
    batch_size=32,
    validation_data=(X_val_def, y_val),  # Especifica los conjuntos de validación
    callbacks=[early_stopping]
)

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Epoch 1/20
2423/2423 ━━━━━━━━━━━━━━━━━━━━ 24s 9ms/step - accuracy: 0.7597 - loss: 0.4785 - val_accuracy: 0.8319 - val_loss: 0.3706
Epoch 2/20
2423/2423 ━━━━━━━━━━━━━━━━━━━━ 20s 8ms/step - accuracy: 0.8275 - loss: 0.3752 - val_accuracy: 0.8432 - val_loss: 0.3601
Epoch 3/20
2423/2423 ━━━━━━━━━━━━━━━━━━━━ 21s 9ms/step - accuracy: 0.8467 - loss: 0.3422 - val_accuracy: 0.8493 - val_loss: 0.3385
Epoch 4/20
2423/2423 ━━━━━━━━━━━━━━━━━━━━ 20s 8ms/step - accuracy: 0.8562 - loss: 0.3232 - val_accuracy: 0.8557 - val_loss: 0.3331
Epoch 5/20
2423/2423 ━━━━━━━━━━━━━━━━━━━━ 20s 8ms/step - accuracy: 0.8662 - loss: 0.3050 - val_accuracy: 0.8572 - val_loss: 0.3535
Epoch 6/20
2423/2423 ━━━━━━━━━━━━━━━━━━━━ 22s 9ms/step - accuracy: 0.8764 - loss: 0.2887 - val_accuracy: 0.8578 - val_loss: 0.3362
Epoch 7/20
2423/2423 ━━━━━━━━━━━━━━━━━━━━ 40s 8ms/step - accuracy: 0.8814 - loss: 0.2725 - val_accuracy: 0.8553 - val_loss: 0.3413
Epoch 8/20
2423/2423 ━━━━━━━━━━━━━━━━━━━━ 21s 9ms/step - accuracy: 0.8894 - loss: 0

In [ ]:

# Evaluación en el conjunto de test
test_loss, test_accuracy = model.evaluate(X_test_def, y_test, batch_size=64)

print(f"Pérdida en el conjunto de test: {test_loss}")
print(f"Precisión en el conjunto de test: {test_accuracy}")

379/379 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.8558 - loss: 0.3334
Pérdida en el conjunto de test: 0.33592358231544495
Precisión en el conjunto de test: 0.8574790358543396


In [ ]:

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Conv1D, MaxPooling1D, Flatten, LSTM, Dense, Dropout, BatchNormalization, Bidirectional
from tensorflow.keras.callbacks import EarlyStopping

# Parámetros
max_len = X_train_def.shape[1]
seed=42
np.random.seed(seed)
random.seed(seed)
tf.random.set_seed(seed)

model = Sequential()
model.add(Embedding(input_dim=num_words, output_dim=embedding_dim, weights=[embedding_matrix_x_train], trainable=False, input_length=max_len))
model.add(Dropout(0.25))

model.add(LSTM(64, return_sequences=True))

model.add(LSTM(64))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))

model.add(Dense(1, activation='sigmoid'))

from tensorflow.keras.optimizers import Adam
optimizer = Adam(learning_rate=0.0005)  # Reducir el learning rate
model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy']) # Función de pérdida binaria

# Definir Early Stopping
early_stopping = EarlyStopping(monitor='val_accuracy', patience=3, restore_best_weights=True)

# Entrenamiento con Early Stopping usando conjuntos de validación específicos
history = model.fit(
    X_train_def, y_train,
    epochs=20,
    batch_size=32,
    validation_data=(X_val_def, y_val),  # Especifica los conjuntos de validación
    callbacks=[early_stopping]
)

Epoch 1/20
2423/2423 ━━━━━━━━━━━━━━━━━━━━ 27s 10ms/step - accuracy: 0.7435 - loss: 0.4955 - val_accuracy: 0.8276 - val_loss: 0.3737
Epoch 2/20
2423/2423 ━━━━━━━━━━━━━━━━━━━━ 21s 9ms/step - accuracy: 0.8268 - loss: 0.3758 - val_accuracy: 0.8392 - val_loss: 0.3654
Epoch 3/20
2423/2423 ━━━━━━━━━━━━━━━━━━━━ 40s 8ms/step - accuracy: 0.8427 - loss: 0.3469 - val_accuracy: 0.8495 - val_loss: 0.3424
Epoch 4/20
2423/2423 ━━━━━━━━━━━━━━━━━━━━ 22s 9ms/step - accuracy: 0.8515 - loss: 0.3300 - val_accuracy: 0.8542 - val_loss: 0.3285
Epoch 5/20
2423/2423 ━━━━━━━━━━━━━━━━━━━━ 41s 9ms/step - accuracy: 0.8625 - loss: 0.3135 - val_accuracy: 0.8586 - val_loss: 0.3316
Epoch 6/20
2423/2423 ━━━━━━━━━━━━━━━━━━━━ 41s 9ms/step - accuracy: 0.8687 - loss: 0.3008 - val_accuracy: 0.8605 - val_loss: 0.3275
Epoch 7/20
2423/2423 ━━━━━━━━━━━━━━━━━━━━ 40s 8ms/step - accuracy: 0.8735 - loss: 0.2902 - val_accuracy: 0.8606 - val_loss: 0.3296
Epoch 8/20
2423/2423 ━━━━━━━━━━━━━━━━━━━━ 21s 8ms/step - accuracy: 0.8785 - loss: 

In [ ]:

# Evaluación en el conjunto de test
test_loss, test_accuracy = model.evaluate(X_test_def, y_test, batch_size=64)

print(f"Pérdida en el conjunto de test: {test_loss}")
print(f"Precisión en el conjunto de test: {test_accuracy}")

379/379 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.8558 - loss: 0.3393
Pérdida en el conjunto de test: 0.3404638171195984
Precisión en el conjunto de test: 0.8583047986030579


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Conv1D, MaxPooling1D, Flatten, LSTM, Dense, Dropout, BatchNormalization, Bidirectional
from tensorflow.keras.callbacks import EarlyStopping

# Parámetros
max_len = X_train_def.shape[1]
seed=42
np.random.seed(seed)
random.seed(seed)
tf.random.set_seed(seed)

model = Sequential()
model.add(Embedding(input_dim=num_words, output_dim=embedding_dim, weights=[embedding_matrix_x_train], trainable=False, input_length=max_len))
model.add(Dropout(0.25))

model.add(LSTM(128, return_sequences=True))
model.add(LSTM(128))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))

model.add(Dense(1, activation='sigmoid'))

from tensorflow.keras.optimizers import Adam
optimizer = Adam(learning_rate=0.0005)  # Reducir el learning rate
model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy']) # Función de pérdida binaria

# Definir Early Stopping
early_stopping = EarlyStopping(monitor='val_accuracy', patience=3, restore_best_weights=True)

# Entrenamiento con Early Stopping usando conjuntos de validación específicos
history = model.fit(
    X_train_def, y_train,
    epochs=20,
    batch_size=32,
    validation_data=(X_val_def, y_val),  # Especifica los conjuntos de validación
    callbacks=[early_stopping]
)

Epoch 1/20
2423/2423 ━━━━━━━━━━━━━━━━━━━━ 26s 10ms/step - accuracy: 0.7559 - loss: 0.4848 - val_accuracy: 0.8332 - val_loss: 0.3678
Epoch 2/20
2423/2423 ━━━━━━━━━━━━━━━━━━━━ 39s 9ms/step - accuracy: 0.8267 - loss: 0.3755 - val_accuracy: 0.8453 - val_loss: 0.3574
Epoch 3/20
2423/2423 ━━━━━━━━━━━━━━━━━━━━ 41s 9ms/step - accuracy: 0.8445 - loss: 0.3429 - val_accuracy: 0.8510 - val_loss: 0.3367
Epoch 4/20
2423/2423 ━━━━━━━━━━━━━━━━━━━━ 41s 9ms/step - accuracy: 0.8562 - loss: 0.3238 - val_accuracy: 0.8548 - val_loss: 0.3284
Epoch 5/20
2423/2423 ━━━━━━━━━━━━━━━━━━━━ 39s 8ms/step - accuracy: 0.8656 - loss: 0.3054 - val_accuracy: 0.8564 - val_loss: 0.3482
Epoch 6/20
2423/2423 ━━━━━━━━━━━━━━━━━━━━ 21s 9ms/step - accuracy: 0.8745 - loss: 0.2895 - val_accuracy: 0.8572 - val_loss: 0.3348
Epoch 7/20
2423/2423 ━━━━━━━━━━━━━━━━━━━━ 21s 8ms/step - accuracy: 0.8799 - loss: 0.2751 - val_accuracy: 0.8528 - val_loss: 0.3391
Epoch 8/20
2423/2423 ━━━━━━━━━━━━━━━━━━━━ 40s 8ms/step - accuracy: 0.8892 - loss: 

In [ ]:

# Evaluación en el conjunto de test
test_loss, test_accuracy = model.evaluate(X_test_def, y_test, batch_size=64)

print(f"Pérdida en el conjunto de test: {test_loss}")
print(f"Precisión en el conjunto de test: {test_accuracy}")

379/379 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.8543 - loss: 0.3321
Pérdida en el conjunto de test: 0.33397775888442993
Precisión en el conjunto de test: 0.8552082777023315


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Conv1D, MaxPooling1D, Flatten, LSTM, Dense, Dropout, BatchNormalization, Bidirectional
from tensorflow.keras.callbacks import EarlyStopping

# Parámetros
max_len = X_train_def.shape[1]
seed=42
np.random.seed(seed)
random.seed(seed)
tf.random.set_seed(seed)

model = Sequential()
model.add(Embedding(input_dim=num_words, output_dim=embedding_dim, weights=[embedding_matrix_x_train], trainable=False, input_length=max_len))
model.add(Dropout(0.25))

model.add(LSTM(256, return_sequences=True))
model.add(LSTM(256))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))

model.add(Dense(1, activation='sigmoid'))

from tensorflow.keras.optimizers import Adam
optimizer = Adam(learning_rate=0.0005)  # Reducir el learning rate
model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy']) # Función de pérdida binaria

# Definir Early Stopping
early_stopping = EarlyStopping(monitor='val_accuracy', patience=3, restore_best_weights=True)

# Entrenamiento con Early Stopping usando conjuntos de validación específicos
history = model.fit(
    X_train_def, y_train,
    epochs=20,
    batch_size=32,
    validation_data=(X_val_def, y_val),  # Especifica los conjuntos de validación
    callbacks=[early_stopping]
)

Epoch 1/20
2423/2423 ━━━━━━━━━━━━━━━━━━━━ 25s 9ms/step - accuracy: 0.7632 - loss: 0.4785 - val_accuracy: 0.8349 - val_loss: 0.3703
Epoch 2/20
2423/2423 ━━━━━━━━━━━━━━━━━━━━ 23s 9ms/step - accuracy: 0.8291 - loss: 0.3736 - val_accuracy: 0.8486 - val_loss: 0.3504
Epoch 3/20
2423/2423 ━━━━━━━━━━━━━━━━━━━━ 41s 9ms/step - accuracy: 0.8472 - loss: 0.3406 - val_accuracy: 0.8562 - val_loss: 0.3324
Epoch 4/20
2423/2423 ━━━━━━━━━━━━━━━━━━━━ 41s 9ms/step - accuracy: 0.8586 - loss: 0.3197 - val_accuracy: 0.8581 - val_loss: 0.3346
Epoch 5/20
2423/2423 ━━━━━━━━━━━━━━━━━━━━ 41s 9ms/step - accuracy: 0.8682 - loss: 0.2987 - val_accuracy: 0.8558 - val_loss: 0.3468
Epoch 6/20
2423/2423 ━━━━━━━━━━━━━━━━━━━━ 41s 9ms/step - accuracy: 0.8797 - loss: 0.2802 - val_accuracy: 0.8559 - val_loss: 0.3585
Epoch 7/20
2423/2423 ━━━━━━━━━━━━━━━━━━━━ 41s 9ms/step - accuracy: 0.8902 - loss: 0.2585 - val_accuracy: 0.8535 - val_loss: 0.3681


In [ ]:

# Evaluación en el conjunto de test
test_loss, test_accuracy = model.evaluate(X_test_def, y_test, batch_size=64)

print(f"Pérdida en el conjunto de test: {test_loss}")
print(f"Precisión en el conjunto de test: {test_accuracy}")

379/379 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.8544 - loss: 0.3358
Pérdida en el conjunto de test: 0.33671441674232483
Precisión en el conjunto de test: 0.855167031288147


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Conv1D, MaxPooling1D, Flatten, LSTM, Dense, Dropout, BatchNormalization, Bidirectional
from tensorflow.keras.callbacks import EarlyStopping

# Parámetros
max_len = X_train_def.shape[1]
seed=42
np.random.seed(seed)
random.seed(seed)
tf.random.set_seed(seed)

model = Sequential()
model.add(Embedding(input_dim=num_words, output_dim=embedding_dim, weights=[embedding_matrix_x_train], trainable=False, input_length=max_len))
model.add(Dropout(0.25))

model.add(LSTM(256, return_sequences=True))
model.add(LSTM(128))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))

model.add(Dense(1, activation='sigmoid'))

from tensorflow.keras.optimizers import Adam
optimizer = Adam(learning_rate=0.0005)  # Reducir el learning rate
model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy']) # Función de pérdida binaria

# Definir Early Stopping
early_stopping = EarlyStopping(monitor='val_accuracy', patience=3, restore_best_weights=True)

# Entrenamiento con Early Stopping usando conjuntos de validación específicos
history = model.fit(
    X_train_def, y_train,
    epochs=20,
    batch_size=32,
    validation_data=(X_val_def, y_val),  # Especifica los conjuntos de validación
    callbacks=[early_stopping]
)

Epoch 1/20
2423/2423 ━━━━━━━━━━━━━━━━━━━━ 25s 9ms/step - accuracy: 0.7590 - loss: 0.4836 - val_accuracy: 0.8344 - val_loss: 0.3686
Epoch 2/20
2423/2423 ━━━━━━━━━━━━━━━━━━━━ 39s 8ms/step - accuracy: 0.8294 - loss: 0.3738 - val_accuracy: 0.8473 - val_loss: 0.3543
Epoch 3/20
2423/2423 ━━━━━━━━━━━━━━━━━━━━ 22s 9ms/step - accuracy: 0.8475 - loss: 0.3397 - val_accuracy: 0.8540 - val_loss: 0.3348
Epoch 4/20
2423/2423 ━━━━━━━━━━━━━━━━━━━━ 41s 9ms/step - accuracy: 0.8583 - loss: 0.3192 - val_accuracy: 0.8559 - val_loss: 0.3369
Epoch 5/20
2423/2423 ━━━━━━━━━━━━━━━━━━━━ 41s 9ms/step - accuracy: 0.8707 - loss: 0.2989 - val_accuracy: 0.8579 - val_loss: 0.3502
Epoch 6/20
2423/2423 ━━━━━━━━━━━━━━━━━━━━ 20s 8ms/step - accuracy: 0.8809 - loss: 0.2780 - val_accuracy: 0.8585 - val_loss: 0.3535
Epoch 7/20
2423/2423 ━━━━━━━━━━━━━━━━━━━━ 22s 9ms/step - accuracy: 0.8902 - loss: 0.2579 - val_accuracy: 0.8556 - val_loss: 0.3570
Epoch 8/20
2423/2423 ━━━━━━━━━━━━━━━━━━━━ 41s 9ms/step - accuracy: 0.9009 - loss: 0

In [ ]:

# Evaluación en el conjunto de test
test_loss, test_accuracy = model.evaluate(X_test_def, y_test, batch_size=64)

print(f"Pérdida en el conjunto de test: {test_loss}")
print(f"Precisión en el conjunto de test: {test_accuracy}")

379/379 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8545 - loss: 0.3554
Pérdida en el conjunto de test: 0.3548884689807892
Precisión en el conjunto de test: 0.8569010496139526


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Conv1D, MaxPooling1D, Flatten, LSTM, Dense, Dropout, BatchNormalization, Bidirectional
from tensorflow.keras.callbacks import EarlyStopping

# Parámetros
max_len = X_train_def.shape[1]
seed=42
np.random.seed(seed)
random.seed(seed)
tf.random.set_seed(seed)

model = Sequential()
model.add(Embedding(input_dim=num_words, output_dim=embedding_dim, weights=[embedding_matrix_x_train], trainable=False, input_length=max_len))
model.add(Dropout(0.25))

model.add(LSTM(128, return_sequences=True))
model.add(LSTM(64))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))

model.add(Dense(1, activation='sigmoid'))

from tensorflow.keras.optimizers import Adam
optimizer = Adam(learning_rate=0.0005)  # Reducir el learning rate
model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy']) # Función de pérdida binaria

# Definir Early Stopping
early_stopping = EarlyStopping(monitor='val_accuracy', patience=3, restore_best_weights=True)

# Entrenamiento con Early Stopping usando conjuntos de validación específicos
history = model.fit(
    X_train_def, y_train,
    epochs=20,
    batch_size=32,
    validation_data=(X_val_def, y_val),  # Especifica los conjuntos de validación
    callbacks=[early_stopping]
)

Epoch 1/20
2423/2423 ━━━━━━━━━━━━━━━━━━━━ 24s 9ms/step - accuracy: 0.7579 - loss: 0.4865 - val_accuracy: 0.8280 - val_loss: 0.3767
Epoch 2/20
2423/2423 ━━━━━━━━━━━━━━━━━━━━ 41s 9ms/step - accuracy: 0.8264 - loss: 0.3756 - val_accuracy: 0.8410 - val_loss: 0.3599
Epoch 3/20
2423/2423 ━━━━━━━━━━━━━━━━━━━━ 41s 9ms/step - accuracy: 0.8451 - loss: 0.3447 - val_accuracy: 0.8497 - val_loss: 0.3414
Epoch 4/20
2423/2423 ━━━━━━━━━━━━━━━━━━━━ 41s 9ms/step - accuracy: 0.8565 - loss: 0.3251 - val_accuracy: 0.8533 - val_loss: 0.3358
Epoch 5/20
2423/2423 ━━━━━━━━━━━━━━━━━━━━ 40s 9ms/step - accuracy: 0.8660 - loss: 0.3069 - val_accuracy: 0.8591 - val_loss: 0.3470
Epoch 6/20
2423/2423 ━━━━━━━━━━━━━━━━━━━━ 41s 9ms/step - accuracy: 0.8748 - loss: 0.2913 - val_accuracy: 0.8593 - val_loss: 0.3302
Epoch 7/20
2423/2423 ━━━━━━━━━━━━━━━━━━━━ 40s 8ms/step - accuracy: 0.8816 - loss: 0.2750 - val_accuracy: 0.8557 - val_loss: 0.3344
Epoch 8/20
2423/2423 ━━━━━━━━━━━━━━━━━━━━ 20s 8ms/step - accuracy: 0.8883 - loss: 0

In [ ]:

# Evaluación en el conjunto de test
test_loss, test_accuracy = model.evaluate(X_test_def, y_test, batch_size=64)

print(f"Pérdida en el conjunto de test: {test_loss}")
print(f"Precisión en el conjunto de test: {test_accuracy}")

379/379 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8548 - loss: 0.3313
Pérdida en el conjunto de test: 0.33376309275627136
Precisión en el conjunto de test: 0.8557863235473633
